In [64]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sp

## Things which are given / can be calculated 

In [65]:
P_d_sp = sp.symbols('P_d')
D_p_sp, D_g_sp = sp.symbols('D_p D_g')
N_p_sp, N_g_sp, N_g_guess_sp = sp.symbols('N_p N_g, N_g_guess') 

n_p_sp, n_g_sp, n_g_guess_sp = sp.symbols('n_p n_g n_g_guess')
VR_sp, VR_guess_sp, v_t_sp = sp.symbols('VR VR_guess v_t')
power_sp, lifetime_sp = sp.symbols('power lifetime')
W_t_sp = sp.symbols('W_t')


#enter given values

uk = sp.nan
all_values = {
    P_d_sp: 12,
    D_p_sp: sp.nan,
    D_g_sp: uk,

    N_p_sp: 20,
    N_g_sp: uk,
    N_g_guess_sp: uk,

    n_p_sp: 1160,
    n_g_sp: uk,
    n_g_guess_sp: 600,
    
    VR_sp: sp.nan,
    VR_guess_sp: sp.nan,
    v_t_sp: sp.nan,
        
    power_sp: 15,
    lifetime_sp: 15000,
    W_t_sp: sp.nan
}


phi_n = 20
psi = 15
phi_t = np.nan
F = 2

# To have in case
# all_values = {
#     P_d: 10,
#     D_p: sp.nan,
#     D_g: sp.nan,
#     C: sp.nan,
#     N_p: 17,
#     N_g: 54,
#     n_p: 1200,
#     n_g: sp.nan,
#     n_g_guess: 387,
#     VR: sp.nan,
#     VR_guess: sp.nan,
#     v_t: sp.nan,
#     F: sp.nan,
#     power: sp.nan,
#     lifetime: sp.nan
# }

all_values, len(all_values)


({P_d: 12,
  D_p: nan,
  D_g: nan,
  N_p: 20,
  N_g: nan,
  N_g_guess: nan,
  n_p: 1160,
  n_g: nan,
  n_g_guess: 600,
  VR: nan,
  VR_guess: nan,
  v_t: nan,
  power: 15,
  lifetime: 15000,
  W_t: nan},
 15)

## Things to find from a table

some, like Y,Z can only be found later, tbd how to handle this

In [66]:
J_p = 0.44
J_G = 0

I = 0.19

C_p = 2300 #for steel

k_o = 1.5
k_s = 1.0

k_B = 1.0
k_v = 1.14
k_R = 1.0


#Need cycles to find, but these are not calculated until later
#im not sure the best way to handle this
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922

#from formula below
k_m = None

#for C_ma
cond = "commercial Enclosed".upper()
# cond = "precision".upper()
# cond = "extra".upper()

safety_factor = 1


## Prelim calculations
The guessing on the output shaft speed is jank. Currently it is set to ceil, but idk if this has negative implications

### Create the equations

In [67]:

eq1 = sp.Eq(VR_guess_sp, n_p_sp / n_g_guess_sp)
eq2 = sp.Eq(N_g_guess_sp, N_p_sp * VR_guess_sp)

eq3 = sp.Eq(VR_sp, N_g_sp / N_p_sp)


eq4 = sp.Eq(n_g_sp, n_p_sp / VR_sp)
eq5 = sp.Eq(D_p_sp, N_p_sp / P_d_sp)
eq6 = sp.Eq(D_g_sp, N_g_sp / P_d_sp)
eq7 = sp.Eq(v_t_sp, np.pi * D_p_sp * n_p_sp / 12)
eq8 = sp.Eq(W_t_sp, 33000 * power_sp / v_t_sp)
eq9 = sp.Eq(VR_sp, n_p_sp / n_g_sp)



## Solve the equations 

In [68]:
#creates a thing of all the non-nan values
known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value


Solve the equations for VR_guess, n_g_guess, N_p, n_g

In [69]:
# Solve equations eq1 and eq2
solutions_eq1_eq2 = sp.solve([eq1, eq2], dict=True)

# Display the solutions
# for solution in solutions_eq1_eq2:
#     for var, expr in solution.items():
#         print(f"{var} = {expr}")


for solution in solutions_eq1_eq2:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values




{P_d: 12,
 N_p: 20,
 n_p: 1160,
 n_g_guess: 600,
 power: 15,
 lifetime: 15000,
 N_g_guess: 116/3,
 VR_guess: 29/15}

In [70]:
a = 0
if known_values[N_g_guess_sp] == sp.nan:  
    a = sp.nan
else:
    a = known_values[N_g_guess_sp]
    known_values[N_g_sp] = round(a, 0)
# # known_values[N_g_sp]

# Solve equation eq3 with the updated known values

if all_values[n_g_guess_sp] == sp.nan:
    pass

else:

    solution_eq3 = sp.solve(eq3.subs(known_values), dict=True)

    # Update known values with the solution of eq3
    for sol in solution_eq3:
        for var, expr in sol.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values, len(known_values), len(all_values)




({P_d: 12,
  N_p: 20,
  n_p: 1160,
  n_g_guess: 600,
  power: 15,
  lifetime: 15000,
  N_g_guess: 116/3,
  VR_guess: 29/15,
  N_g: 39.0000000000000,
  VR: 1.95000000000000},
 10,
 15)

#### Solve the equations multiple times bc some of them depend on others

In [71]:
# Iteratively solve equations until all variables are found

run_count = 0
while True:

    previous_known_values = known_values.copy()
    
    # Solve equations with current known values
    solutions = sp.solve([eq4, eq5.subs(known_values), eq6.subs(known_values), eq7.subs(known_values), eq8.subs(known_values), eq9.subs(known_values)], dict=True)

    # Update known values with solutions
    print(len(solutions))
    for solution in solutions:

        for var, expr in solution.items():

            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)    

    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1
        print(f"Run interation: {run_count}")

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


1
Run interation: 1
0
P_d = 12
N_p = 20
n_p = 1160
n_g_guess = 600
power = 15
lifetime = 15000
N_g_guess = 116/3
VR_guess = 29/15
N_g = 39.0000000000000
VR = 1.95000000000000
D_g = 3.25000000000000
D_p = 1.66666666666667
W_t = 977.979684792268
n_g = 594.871794871795
v_t = 506.145483078355


#### Assign each variable a value, eventually these should all be numbers

In [72]:
# assert(len(known_values) == len(all_values))
len(known_values), len(all_values)
# known_values

for var, value in known_values.items():
    all_values[var] = value


P_d = all_values[P_d_sp]
D_p = all_values[D_p_sp]
D_g = all_values[D_g_sp]
N_p = all_values[N_p_sp]
N_g = all_values[N_g_sp]
n_p = all_values[n_p_sp]
n_g = all_values[n_g_sp]
n_g_guess = all_values[n_g_guess_sp]
VR = all_values[VR_sp]
VR_guess = all_values[VR_guess_sp]
v_t = all_values[v_t_sp]
power = all_values[power_sp]
lifetime = all_values[lifetime_sp]
W_t = all_values[W_t_sp]


all_values




{P_d: 12,
 D_p: 1.66666666666667,
 D_g: 3.25000000000000,
 N_p: 20,
 N_g: 39.0000000000000,
 N_g_guess: 116/3,
 n_p: 1160,
 n_g: 594.871794871795,
 n_g_guess: 600,
 VR: 1.95000000000000,
 VR_guess: 29/15,
 v_t: 506.145483078355,
 power: 15,
 lifetime: 15000,
 W_t: 977.979684792268}

### Final assorted Calculations

#### Calculate angles

In [73]:
phi_t = np.degrees(np.arctan(np.tan(np.radians(phi_n)) / np.cos(np.radians(psi))))

phi_n, phi_t, psi



(20, 20.64689648704647, 15)

#### Pitches there may be smth wrong with this

In [74]:
###

P_nd = P_d / np.cos(np.deg2rad(psi))
P_x = np.pi / (P_d * np.cos(np.deg2rad(psi)))

P_nd, P_x

(12.423314164920995, 0.2710346702344015)

#### Face Value

In [75]:
if F is np.nan:
    F = 2 * P_x

F

2

#### Center distance

In [76]:

C = (N_p + N_g) / (2 * P_d)
print(f"C = {C} inches")

C = 2.45833333333333 inches


#### pitch line speed

In [77]:
v_t = np.pi * D_p * n_p / 12
print(f"Pitch line velocity: {v_t:.2f} ft/min")

Pitch line velocity: 506.15 ft/min


### Calculating Forces

In [78]:
W_t = 33000 * power / v_t
W_r = W_t * np.tan(phi_t * np.pi / 180)
Wx = W_t * np.tan(psi * np.pi / 180)

W_t, W_r, Wx
print(f"W_t = {W_t}, W_r = {W_r}, Wx = {Wx}")

W_t = 977.979684792267, W_r = 368.512245240354, Wx = 262.048866754132


## Calulating correction Factors

### Calculate $C_{pf}$

In [79]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf

0.107500000000000

### Calculate $C_{ma}$

In [80]:
#C_ma

assert(cond is not None)

C_ma = None

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

assert(C_ma is not None)
C_ma

0.15816280000000002

In [81]:
k_m = 1 + C_ma + C_pf
k_m

1.26566280000000

## Loading Cycles $N_{cp}, N_{cg}$

In [82]:
# Number of cycles

N_cp = 60 * lifetime * n_p
N_cg = 60 * lifetime * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")
# N_cp, N_cg

N_cp: 1.04e+09, N_cg: 5.35e+8


### *** Implement $Y_{xx}$  and $Z_{xx}$

In [83]:

Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922


## Stresses and allowable Stresses

### pinion bending stress number

In [93]:
s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
print(f"s_tp = {s_tp} psi")

s_tp = 28863.0707126916 psi


In [85]:
s_tG = s_tp * J_p / J_G
s_tG

zoo

### allowbable bending stress number

In [86]:
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

30382.1796975701

In [87]:
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than

zoo

### Calc $S_c$

In [88]:
S_c = C_p * np.sqrt(  float( (W_t * k_o * k_s * k_m  * k_v) / (F * D_p * I) ) ) #
S_c

132963.8686486521

### FInd adjusted values of $S_c$

In [89]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
S_acP_Greater_than

148066.6688737774

In [90]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
S_acg_Greater_than

144212.4388814014

## Brinell Hardness Calculations

In [91]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

Contact_HB_p: 369.46 psi 
Bending_HB_p: 227.45 psi



TypeError: unsupported format string passed to ComplexInfinity.__format__

### use tables to find the appropriate Material, and enter its brinell hardness

In [ ]:
HB = None

### Calculate actaul allowed bending and contact stresses

In [ ]:

S_acp = None
S_atp = None

S_acg = None
S_atg = None


if int(HB_grade) == 1:
    S_acp = (HB * 0.322 + 29.1 ) * 1000 
    S_atp = (HB * 0.0773 + 12.8 ) * 1000

    S_acg = (HB * 0.322 + 29.1 ) * 1000
    S_atg = (HB * 0.0773 + 12.8 ) * 1000

elif int(HB_grade) == 2:

    S_acp = (HB * 0.349 + 34.3 ) * 1000
    S_atp = (HB * 0.102 + 16.4 ) * 1000

    S_acg = (HB * 0.349 + 34.3 ) * 1000
    S_atg = (HB * 0.102 + 16.4 ) * 1000
    
else:
    assert(False)

print(f"S_acp = {S_acp}, S_atp = {S_atp}, S_acg = {S_acg}, S_atg = {S_atg}")


### Calculate Safety factor

In [ ]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")



## Power transmitting Capacity

In [ ]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")